In [1]:
import numpy as np
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error
import random

In [2]:
class SimpleRLAgent:
    def __init__(self):
        # Q-table with 8 syndromes as keys and actions (0: do nothing, 1: flip q0, 2: flip q1, 3: flip q2)
        self.q_table = {format(i, '03b'): random.choice([0, 1, 2, 3]) for i in range(8)}

    def act(self, syndrome):
        return self.q_table.get(syndrome, 0)

In [3]:
def apply_depolarizing_noise(qc, qubits, error_prob):
    noise_model = NoiseModel()
    dep_error_1q = depolarizing_error(error_prob, 1)
    dep_error_2q = depolarizing_error(error_prob, 2)

    for q in qubits:
        noise_model.add_quantum_error(dep_error_1q, ["u1", "u2", "u3", "id"], [q])

    for q1, q2 in [(0, 1), (0, 2)]:
        noise_model.add_quantum_error(dep_error_2q, ["cx"], [q1, q2])

    return noise_model

In [4]:
def create_qec_circuit():
    qr = QuantumRegister(3, 'q')
    cr = ClassicalRegister(3, 'c')
    qc = QuantumCircuit(qr, cr)

    # Encoding |ψ⟩ → |ψψψ⟩
    qc.h(0)
    qc.cx(0, 1)
    qc.cx(0, 2)

    return qc, qr, cr

In [5]:
def add_decoding_and_measurements(qc, qr, cr):
    # Syndrome decoding
    qc.cx(0, 1)
    qc.cx(0, 2)
    qc.measure([0, 1, 2], [0, 1, 2])
    return qc

In [6]:
def simulate_circuit(qc, noise_model=None, shots=1024):
    simulator = AerSimulator()
    result = simulator.run(qc, noise_model=noise_model, shots=shots).result()
    counts = result.get_counts()
    return counts

In [7]:
def simulate_qec_with_noise(noise_prob=0.01, shots=1024):
    qc, qr, cr = create_qec_circuit()
    noise_model = apply_depolarizing_noise(qc, [0, 1, 2], noise_prob)
    qc = add_decoding_and_measurements(qc, qr, cr)
    return simulate_circuit(qc, noise_model=noise_model, shots=shots)

In [8]:
def simulate_qec_without_noise(shots=1024):
    qc, qr, cr = create_qec_circuit()
    qc = add_decoding_and_measurements(qc, qr, cr)
    return simulate_circuit(qc, noise_model=None, shots=shots)

In [9]:
def apply_rl_agent(agent, counts):
    corrections = {}
    for outcome in counts:
        action = agent.act(outcome)
        corrections[outcome] = action
    return corrections

In [10]:
if __name__ == "__main__":
    noise_probability = 0.05
    results_with_noise = simulate_qec_with_noise(noise_probability)
    results_without_noise = simulate_qec_without_noise()

    print("\nMeasurement Outcomes WITH noise and correction:")
    print(results_with_noise)

    print("\nMeasurement Outcomes WITHOUT noise:")
    print(results_without_noise)

    rl_agent = SimpleRLAgent()
    rl_decisions = apply_rl_agent(rl_agent, results_with_noise)

    print("\nRL Agent Decisions for Each Syndrome:")
    for syndrome, action in rl_decisions.items():
        print(f"Syndrome: {syndrome} -> Action: {action}")


Measurement Outcomes WITH noise and correction:
{'111': 13, '011': 30, '110': 15, '001': 417, '100': 35, '101': 27, '010': 29, '000': 458}

Measurement Outcomes WITHOUT noise:
{'000': 491, '001': 533}

RL Agent Decisions for Each Syndrome:
Syndrome: 111 -> Action: 1
Syndrome: 011 -> Action: 1
Syndrome: 110 -> Action: 2
Syndrome: 001 -> Action: 0
Syndrome: 100 -> Action: 1
Syndrome: 101 -> Action: 1
Syndrome: 010 -> Action: 1
Syndrome: 000 -> Action: 2
